In [1]:
import numpy as np
import time

# Naive Matrix Multiplication
def naive_matrix_multiplication(A, B):
    n = A.shape[0]
    C = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
    return C

# Blocked Matrix Multiplication
def blocked_matrix_multiplication(A, B, block_size):
    n = A.shape[0]
    C = np.zeros((n, n))
    for i in range(0, n, block_size):
        for j in range(0, n, block_size):
            for k in range(0, n, block_size):
                # Multiply the sub-blocks
                for ii in range(i, min(i + block_size, n)):
                    for jj in range(j, min(j + block_size, n)):
                        for kk in range(k, min(k + block_size, n)):
                            C[ii][jj] += A[ii][kk] * B[kk][jj]
    return C

# Generate random matrices for testing
n = 512  # Matrix size
block_size = 64  # Block size for blocking optimization
A = np.random.rand(n, n)
B = np.random.rand(n, n)

# Measure performance of naive multiplication
start_time = time.time()
C_naive = naive_matrix_multiplication(A, B)
naive_time = time.time() - start_time
print(f"Naive Matrix Multiplication Time: {naive_time:.2f} seconds")

# Measure performance of blocked multiplication
start_time = time.time()
C_blocked = blocked_matrix_multiplication(A, B, block_size)
blocked_time = time.time() - start_time
print(f"Blocked Matrix Multiplication Time: {blocked_time:.2f} seconds")

Naive Matrix Multiplication Time: 178.02 seconds
Blocked Matrix Multiplication Time: 152.52 seconds


Implement HPC

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from scipy.sparse import random as sparse_random
from scipy.sparse.linalg import cg  # Conjugate Gradient solver

# Set matrix dimensions and sparsity
matrix_size = 1000  # You can scale this as needed
density = 0.01  # Sparsity of the matrix

# Generate a large random sparse matrix
A = sparse_random(matrix_size, matrix_size, density=density, format='csr', dtype=np.float32)
A = A @ A.T  # Make it symmetric positive-definite
b = np.random.rand(matrix_size).astype(np.float32)  # Random vector for Ax = b

# Solve with traditional PCG for comparison
x_pcg, exit_code = cg(A, b)

# 3. Neural Network-Based Surrogate Model
# Define the neural network model using TensorFlow/Keras
def build_surrogate_model(input_shape):
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(matrix_size)  # Output layer: approximate solution vector x
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Build the surrogate model
input_shape = (matrix_size,)  # Each input vector b has size 'matrix_size'
model = build_surrogate_model(input_shape)

# 4. Generate training data
def generate_training_data(num_samples):
    b_vectors = []
    x_solutions = []

    for _ in range(num_samples):
        # Generate synthetic data
        A = sparse_random(matrix_size, matrix_size, density=density, format='csr', dtype=np.float32)
        A = A @ A.T  # Make it symmetric positive-definite
        b = np.random.rand(matrix_size).astype(np.float32)

        # Solve using the PCG solver to get ground truth solution
        x, _ = cg(A, b)  # Ground truth solution

        b_vectors.append(b)  # Input to the model
        x_solutions.append(x)  # Expected output (solution)

    return np.array(b_vectors), np.array(x_solutions)

# Generate training data
num_samples = 1000
b_train, x_train = generate_training_data(num_samples)

# Train the model
history = model.fit(b_train, x_train, epochs=50, batch_size=32)

# 5. Evaluate Performance
# Evaluate the surrogate model's performance
def evaluate_surrogate_model(b):
    # Use the surrogate model to predict the solution
    x_pred = model.predict(b.reshape(1, -1))  # Reshape for batch prediction
    return x_pred

# Evaluate on a test matrix
A_test = sparse_random(matrix_size, matrix_size, density=density, format='csr', dtype=np.float32)
A_test = A_test @ A_test.T
b_test = np.random.rand(matrix_size).astype(np.float32)

# Traditional PCG solution
x_pcg_test, _ = cg(A_test, b_test)

# Surrogate model solution
x_pred_test = evaluate_surrogate_model(b_test)

# 6. Calculate and print performance improvement
import time
start_time_pcg = time.time()
x_pcg_test, _ = cg(A_test, b_test)
end_time_pcg = time.time()
pcg_time = end_time_pcg - start_time_pcg

start_time_surrogate = time.time()
x_pred_test = evaluate_surrogate_model(b_test)
end_time_surrogate = time.time()
surrogate_time = end_time_surrogate - start_time_surrogate

print(f"PCG solver time: {pcg_time} seconds")
print(f"Surrogate model time: {surrogate_time} seconds")
print(f"Performance improvement: {pcg_time / surrogate_time}x")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_isolve/iterative.py:418: RuntimeWarning: overflow encountered in add
  x += alpha*p
/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_isolve/iterative.py:410: RuntimeWarning: overflow encountered in multiply
  p *= beta


Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: nan
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: nan
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: nan
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: nan
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: nan
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: nan
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: nan
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: nan
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: nan
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: nan
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: nan
Epoch 12/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: nan
Epoch 13/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: nan
Epoch 14/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: nan
Epoch 15/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: nan
Epoch 16/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/st